In [1]:
!git clone https://github.com/co1dtype/wildfire_CD1.git

Cloning into 'wildfire_CD1'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 62 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), 30.74 KiB | 1.40 MiB/s, done.


In [1]:
%cd wildfire_CD1
!python train.py

/home/hyj/ChanHyung/Image_segementation/WildFire_Detection/wildfire_CD1
Traceback (most recent call last):
  File "/home/hyj/ChanHyung/Image_segementation/WildFire_Detection/wildfire_CD1/train.py", line 6, in <module>
    from modules.trainer import LightningModel
  File "/home/hyj/ChanHyung/Image_segementation/WildFire_Detection/wildfire_CD1/modules/trainer.py", line 8, in <module>
    from modules.optimizers import get_optimizer
ModuleNotFoundError: No module named 'modules.optimizers'
